In [1]:
import os
import seaborn as sns
import nibabel as nib
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

In [2]:
def softmax(x, T):
    x = x / T
    f = np.exp(x - np.max(x, axis = 0))  # shift values
    return f / f.sum(axis = 0)

def ComputMetric(ACTUAL, PREDICTED):
    ACTUAL = ACTUAL.flatten()
    PREDICTED = PREDICTED.flatten()
    idxp = ACTUAL == True
    idxn = ACTUAL == False

    tp = np.sum(ACTUAL[idxp] == PREDICTED[idxp])
    tn = np.sum(ACTUAL[idxn] == PREDICTED[idxn])
    fp = np.sum(idxn) - tn
    fn = np.sum(idxp) - tp
    FPR = fp / (fp + tn)
    if tp == 0 :
        dice = 0
        Precision = 0
        Sensitivity = 0
    else:
        dice = 2 * tp / (2 * tp + fp + fn)
        Precision = tp / (tp + fp)
        Sensitivity = tp / (tp + fn)
    return dice, Sensitivity, Precision

In [3]:
def sum_tensor(inp, axes, keepdim=False):
    axes = np.unique(axes).astype(int)
    if keepdim:
        for ax in axes:
            inp = inp.sum(int(ax), keepdim=True)
    else:
        for ax in sorted(axes, reverse=True):
            inp = inp.sum(int(ax))
    return inp

def get_tp_fp_fn(net_output, gt, axes=None, mask=None, square=False):
    """
    net_output must be (b, c, x, y(, z)))
    gt must be a label map (shape (b, 1, x, y(, z)) OR shape (b, x, y(, z))) or one hot encoding (b, c, x, y(, z))
    if mask is provided it must have shape (b, 1, x, y(, z)))
    :param net_output:
    :param gt:
    :param axes:
    :param mask: mask must be 1 for valid pixels and 0 for invalid pixels
    :param square: if True then fp, tp and fn will be squared before summation
    :return:
    """
    if axes is None:
        axes = tuple(range(2, len(net_output.size())))

    shp_x = net_output.shape
    shp_y = gt.shape

    with torch.no_grad():
        if len(shp_x) != len(shp_y):
            gt = gt.view((shp_y[0], 1, *shp_y[1:]))

        if all([i == j for i, j in zip(net_output.shape, gt.shape)]):
            # if this is the case then gt is probably already a one hot encoding
            y_onehot = gt
        else:
            gt = gt.long()
            y_onehot = torch.zeros(shp_x)
            if net_output.device.type == "cuda":
                y_onehot = y_onehot.cuda(net_output.device.index)
            y_onehot.scatter_(1, gt, 1)

    tp = net_output * y_onehot
    fp = net_output * (1 - y_onehot)
    fn = (1 - net_output) * y_onehot

    if mask is not None:
        tp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(tp, dim=1)), dim=1)
        fp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fp, dim=1)), dim=1)
        fn = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fn, dim=1)), dim=1)

    if square:
        tp = tp ** 2
        fp = fp ** 2
        fn = fn ** 2

    tp = sum_tensor(tp, axes, keepdim=False)
    fp = sum_tensor(fp, axes, keepdim=False)
    fn = sum_tensor(fn, axes, keepdim=False)

    return tp, fp, fn


def SoftDiceLoss(x, y, loss_mask = None, smooth = 1e-5, batch_dice = False):
    '''
    Batch_dice means that we want to calculate the dsc of all batch
    It would make more sense for small patchsize, aka DeepMedic based training.
    '''
    shp_x = x.shape
    square = False

    axes = [0] + list(range(2, len(shp_x)))
    tp, fp, fn = get_tp_fp_fn(x, y, axes, loss_mask, square)
    dc = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    dc_process = dc

    return dc_process

In [4]:
resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostateval/'
prostatevalpath = '/vol/biomedic3/zl9518/Prostatedata/datafiletestB/'

In [5]:
DatafiletsImgc1 = prostatevalpath + 'seg-eval.txt'
Imgfiletsc1 = open(DatafiletsImgc1)
Imgreadc1 = Imgfiletsc1.read().splitlines()

In [6]:
probs_all = []
preds_class_all = []
targets_all = []
for Imgnamec1 in tqdm(Imgreadc1):
    knamelist = Imgnamec1.split("/")
    kname = knamelist[-1][0:6]
    
    cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
    cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
    cls0read = nib.load(cls0filename)
    cls1read = nib.load(cls1filename)
    cls0logit = cls0read.get_fdata()
    cls1logit = cls1read.get_fdata()
    GTread = nib.load(Imgnamec1)
    GTimg = GTread.get_fdata()
    imgshape = GTimg.shape
    cls0flatten = cls0logit.flatten()
    cls1flatten = cls1logit.flatten()
    clsflatten = np.stack((cls0flatten, cls1flatten))
    GTflatten = GTimg.flatten()
    probflatten = softmax(clsflatten, T = 1.6)
    
    pred_class = np.argmax(probflatten, axis = 0)
    preds_class_all = np.concatenate((preds_class_all, pred_class), axis=0)
    targets_all = np.concatenate((targets_all, GTflatten), axis=0)
    probmax = np.max(probflatten, axis = 0)
    probs_all = np.concatenate((probs_all, probmax), axis=0)


100%|█████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.24it/s]


In [7]:
print(np.sum(targets_all == preds_class_all) / len(targets_all))
print(np.mean(probs_all))

0.9960332029006206
0.9960515118056786


In [33]:
# probs_all_c0 = []
# preds_class_all_c0 = []
# targets_all_c0 = []
# probs_all_c1 = []
# preds_class_all_c1 = []
# targets_all_c1 = []
softDSCs = []
realDSCs = []
for Imgnamec1 in tqdm(Imgreadc1):
    knamelist = Imgnamec1.split("/")
    kname = knamelist[-1][0:6]
    
    cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
    cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
    cls0read = nib.load(cls0filename)
    cls1read = nib.load(cls1filename)
    cls0logit = cls0read.get_fdata()
    cls1logit = cls1read.get_fdata()
    GTread = nib.load(Imgnamec1)
    GTimg = GTread.get_fdata()
    imgshape = GTimg.shape
    cls0flatten = cls0logit.flatten()
    cls1flatten = cls1logit.flatten()
    clsflatten = np.stack((cls0flatten, cls1flatten))
    GTflatten = GTimg.flatten()
    probflatten = softmax(clsflatten, T = 1.0)
    
    preds_all_argmax = np.argmax(clsflatten, axis = 0)
    # for cls 0, BG class 
#     targets_y1 = np.where(preds_all_argmax==0)[0]
#     probflatten_c0 = softmax(clsflatten[:, targets_y1], T = 1.6)
#     pred_class_c0 = np.argmax(probflatten_c0, axis = 0)
#     preds_class_all_c0 = np.concatenate((preds_class_all_c0, pred_class_c0), axis=0)
    
#     targets_all_c0 = np.concatenate((targets_all_c0, GTflatten[targets_y1]), axis=0)
#     probmax_c0 = np.max(probflatten_c0, axis = 0)
#     probs_all_c0 = np.concatenate((probs_all_c0, probmax_c0), axis=0)
    
#     # for cls 1, FG class
#     targets_y1 = np.where(preds_all_argmax==1)[0]
#     probflatten_c1 = softmax(clsflatten[:, targets_y1], T = 1.7)
#     pred_class_c1 = np.argmax(probflatten_c1, axis = 0)
#     preds_class_all_c1 = np.concatenate((preds_class_all_c1, pred_class_c1), axis=0)
    
#     targets_all_c1 = np.concatenate((targets_all_c1, GTflatten[targets_y1]), axis=0)
#     probmax_c1 = np.max(probflatten_c1, axis = 0)
#     probs_all_c1 = np.concatenate((probs_all_c1, probmax_c1), axis=0)
    
    # for cls 0, BG class 
    targets_y1 = np.where(preds_all_argmax==0)[0]
    probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.8)
    # for cls 1, FG class
    targets_y1 = np.where(preds_all_argmax==1)[0]
    # 1.7
    probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.3)
    probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
    probr_tensor = torch.tensor(probr[np.newaxis, ...])
    GTimgf = np.argmax(probr, axis = 0)
    GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
    
    softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
    softDSCs.append(softDSC[1].numpy())

    realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
    realDSCs.append(realDSC)
    
#     softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
#     softDSCs.append(softDSC[0].numpy())

#     realDSC, _, _ = ComputMetric(1-GTimg, 1-np.argmax(probr, axis = 0))
#     realDSCs.append(realDSC)


100%|█████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.17it/s]


In [34]:
# print(np.sum(targets_all_c0 == preds_class_all_c0) / len(targets_all_c0))
# print(np.mean(probs_all_c0))
# print(np.sum(targets_all_c1 == preds_class_all_c1) / len(targets_all_c1))
# print(np.mean(probs_all_c1))
print(np.mean(softDSCs))
print(np.mean(realDSCs))

0.8927934005334531
0.8925794874268455


In [62]:
# 84 conditions
softDSCs_AC = []
softDSCs_TS = []
softDSCs_CSTS = []
realDSCs = []
for kcon in tqdm(range(1, 84)):
    softDSC_FG_AC = []
    softDSC_FG_TS = []
    softDSC_FG_CSTS = []
    realDSC_FG = []
    resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostattestcondition_' + str(kcon) + '/'
    for Imgnamec1 in Imgreadc1:
        knamelist = Imgnamec1.split("/")
        kname = knamelist[-1][0:6]

        cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
        cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
        cls0read = nib.load(cls0filename)
        cls1read = nib.load(cls1filename)
        cls0logit = cls0read.get_fdata()
        cls1logit = cls1read.get_fdata()
        GTread = nib.load(Imgnamec1)
        GTimg = GTread.get_fdata()
        imgshape = GTimg.shape
        cls0flatten = cls0logit.flatten()
        cls1flatten = cls1logit.flatten()
        clsflatten = np.stack((cls0flatten, cls1flatten))
        GTflatten = GTimg.flatten()
        # By AC
        probflatten = softmax(clsflatten, T = 1.0)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_AC.append(softDSC[1].numpy())
        # By TS
        probflatten = softmax(clsflatten, T = 1.6)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_TS.append(softDSC[1].numpy())
        # By CSTS
        preds_all_argmax = np.argmax(clsflatten, axis = 0)
        # for cls 0, BG class 
        targets_y1 = np.where(preds_all_argmax==0)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.6)
        # for cls 1, FG class
        targets_y1 = np.where(preds_all_argmax==1)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.9)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_CSTS.append(softDSC[1].numpy())

        realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
        realDSC_FG.append(realDSC)
        
    softDSCs_AC.append(np.mean(np.array(softDSC_FG_AC)))
    softDSCs_TS.append(np.mean(np.array(softDSC_FG_TS)))
    softDSCs_CSTS.append(np.mean(np.array(softDSC_FG_CSTS)))
    realDSCs.append(np.mean(np.array(realDSC_FG)))

100%|███████████████████| 83/83 [17:52<00:00, 12.92s/it]


In [63]:
print('AC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_AC))))
print('TS_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_TS))))
print('CSTS_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_CSTS))))

AC_results:
8.682389850061455e-02
TS_results:
0.03662467820391148
CSTS_results:
0.030188914963328883


In [64]:
print(realDSCs)

[0.8457301025307677, 0.870852084028978, 0.828521585768393, 0.790703746725564, 0.7586812862096527, 0.7557484783931, 0.8172327417988589, 0.6604643235449457, 0.8391672071049449, 0.629645911710675, 0.8541350026276865, 0.8736883469998509, 0.8538392221470215, 0.843655497285545, 0.8472068280655369, 0.8415826376847552, 0.8719698924620335, 0.8669337478504607, 0.8188883956186596, 0.8289616381345647, 0.7512113795312175, 0.7984512037339606, 0.8258757147417111, 0.7808367447866433, 0.8216387475687809, 0.7407959047695183, 0.8082450924634974, 0.7488012437878321, 0.8813964100757584, 0.8449041600923041, 0.8723101712065781, 0.7438437175087971, 0.7690050430372952, 0.8036237249343758, 0.6054180714311662, 0.594775255065044, 0.8655259284098629, 0.8193493642947048, 0.8471036691597765, 0.8195559172815617, 0.8942332191332024, 0.886877812107558, 0.8949294033819303, 0.8655104773584853, 0.8947390346757661, 0.8509641068080747, 0.8916376091423593, 0.8932068281575198, 0.8886573243066662, 0.894043014237129, 0.88414099

In [65]:
print(softDSCs_CSTS)

[0.8702601597709705, 0.5146653097008377, 0.8480215327272811, 0.5908085240964449, 0.8264001936756081, 0.6342678277254019, 0.8345615597667864, 0.5867774324229249, 0.85772956165116, 0.6167862937446114, 0.8521182339839264, 0.8575666788241805, 0.8527545252174573, 0.8428973075397586, 0.8417549487698229, 0.8349566964011158, 0.8578925801637183, 0.8584899804114092, 0.8267156726489603, 0.8333627412373723, 0.8031375233899904, 0.804884168253175, 0.7016966360899005, 0.7076394225892692, 0.6931981986187961, 0.7069555182484295, 0.6910189719938, 0.6990766821542791, 0.884241670106767, 0.8724038918449899, 0.881455116354347, 0.8079843958477673, 0.8019315536063948, 0.8742810081639518, 0.7909066784034626, 0.7881487166712728, 0.8819808030414004, 0.8398840819676755, 0.8476036888578763, 0.8749042700385592, 0.8943149119064289, 0.8811824274440486, 0.8971423201322979, 0.8743659631943549, 0.8963994944678092, 0.8730522898548019, 0.8873102334621038, 0.8907318155979626, 0.8830484328724968, 0.8927361059143235, 0.87890

In [66]:
print(softDSCs_TS)

[0.8835471295601198, 0.5258335863516532, 0.8602712728706216, 0.6040689439126108, 0.8387758084349235, 0.6488667443956512, 0.8456610477582845, 0.6014554518893156, 0.8679129574772471, 0.6320750203002559, 0.8657178181548257, 0.8710232990631166, 0.8666605127123574, 0.8567054651659335, 0.8557832475513896, 0.8487327174578325, 0.8713144750506485, 0.8719290850322177, 0.8404998610537417, 0.8471405132886558, 0.8174492827650279, 0.8189720914299443, 0.714555385273155, 0.7209860308769293, 0.7061699634392125, 0.7208145823338781, 0.7041825529076107, 0.7127840956294553, 0.8978366338835755, 0.8863333634773973, 0.8951645034481196, 0.8221395568237708, 0.8156616965187382, 0.8886309350575242, 0.8054426054643553, 0.80327195186624, 0.8957340932524286, 0.8541531310847461, 0.861875316462742, 0.8890754977267946, 0.9078407384788715, 0.8947121365690469, 0.9106804102357661, 0.8880808734255303, 0.909963748066318, 0.8869371628147238, 0.9008249236473975, 0.9042523766369943, 0.8965621530310551, 0.9062604532972527, 0.89

In [67]:
print(softDSCs_AC)

[0.9445208141411179, 0.5600351926555271, 0.931730127079436, 0.6528295465927425, 0.9158539060827395, 0.7136134168357384, 0.9246074382786308, 0.6644763539333387, 0.9389497001221697, 0.7066098338452236, 0.9275612026510706, 0.9325569574687153, 0.9328646004932912, 0.9269801658772193, 0.9270234060337872, 0.9238830251708064, 0.9326770866378464, 0.9336464948378751, 0.9172174173486736, 0.9205507685038764, 0.9035785475003083, 0.9033867852464283, 0.7677991683602265, 0.77648857500991, 0.7670021114181449, 0.7822310609348627, 0.7681708070921458, 0.7782648221080334, 0.9547345456520384, 0.9456113594412106, 0.9535450027183474, 0.9004201920300152, 0.8979303404342277, 0.9448680173310867, 0.8859718451761012, 0.8879338421392813, 0.9540584116065987, 0.9288409827503431, 0.9366174510538441, 0.9459088235601767, 0.9619089739592205, 0.9532975605133911, 0.9635967070577143, 0.9466395431589174, 0.9636146188932045, 0.9455708263766514, 0.9578893452463093, 0.9599190227327112, 0.9549463403474387, 0.961062565393781, 0.9

In [35]:
# 5 conditions
domainlist = ['A', 'C', 'D', 'E', 'F']
softDSCs_AC = []
softDSCs_TS = []
softDSCs_CSTS = []
realDSCs = []
for kcon in tqdm(domainlist):
    softDSC_FG_AC = []
    softDSC_FG_TS = []
    softDSC_FG_CSTS = []
    realDSC_FG = []
    resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostattestcondition_' + kcon + '/'
    
    prostatevalpath = '/vol/biomedic3/zl9518/Prostatedata/datafiletest' + kcon + '/'
    DatafiletsImgc1 = prostatevalpath + 'seg-eval.txt'
    Imgfiletsc1 = open(DatafiletsImgc1)
    Imgreadc1 = Imgfiletsc1.read().splitlines()
    
    for Imgnamec1 in Imgreadc1:
        knamelist = Imgnamec1.split("/")
        kname = knamelist[-1][0:6]

        cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
        cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
        cls0read = nib.load(cls0filename)
        cls1read = nib.load(cls1filename)
        cls0logit = cls0read.get_fdata()
        cls1logit = cls1read.get_fdata()
        GTread = nib.load(Imgnamec1)
        GTimg = GTread.get_fdata()
        imgshape = GTimg.shape
        cls0flatten = cls0logit.flatten()
        cls1flatten = cls1logit.flatten()
        clsflatten = np.stack((cls0flatten, cls1flatten))
        GTflatten = GTimg.flatten()
        # By AC
        probflatten = softmax(clsflatten, T = 1.0)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_AC.append(softDSC[1].numpy())
        # By TS
        probflatten = softmax(clsflatten, T = 1.6)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_TS.append(softDSC[1].numpy())
        # By CSTS
        preds_all_argmax = np.argmax(clsflatten, axis = 0)
        # for cls 0, BG class 
        targets_y1 = np.where(preds_all_argmax==0)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.8)
        # for cls 1, FG class
        targets_y1 = np.where(preds_all_argmax==1)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.3)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_CSTS.append(softDSC[1].numpy())

        realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
        realDSC_FG.append(realDSC)
        
    softDSCs_AC.append(np.mean(np.array(softDSC_FG_AC)))
    softDSCs_TS.append(np.mean(np.array(softDSC_FG_TS)))
    softDSCs_CSTS.append(np.mean(np.array(softDSC_FG_CSTS)))
    realDSCs.append(np.mean(np.array(realDSC_FG)))

100%|███████████████████████████████████████████████████████████████████| 5/5 [00:47<00:00,  9.44s/it]


In [36]:
print('AC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_AC))))
print('TS_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_TS))))
print('CSTS_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_CSTS))))

AC_results:
0.18734415411981173
TS_results:
0.09247324744821413
CSTS_results:
0.07853095452815757


In [69]:
print(realDSCs)

[6.831820639295538e-01, 0.6361422807072967, 0.6095476370292467, 0.8351399018424807, 0.7657602931920688]


In [70]:
print(softDSCs_TS)

[0.807022927996315, 0.7456312568567052, 0.7655065414273087, 0.8600237527812303, 0.8139539348801582]


In [71]:
print(softDSCs_CSTS)

[0.7898991981831605, 0.7305824462124744, 0.7515516465391919, 0.8458572287013559, 0.799992836418788]


In [72]:
print(softDSCs_AC)

[0.8903888833710243, 0.8646945640943136, 0.8701191490312503, 0.9339760285393177, 0.9073143222637996]
